In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate

/Users/maruthienugula/RAG_learnings/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [3]:
#  Loading  and chunking  the document
loader = TextLoader("langchain-rag-dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)
chunks

[Document(metadata={'source': 'langchain-rag-dataset.txt'}, page_content='LangChain is an open-source framework designed to simplify the development of applications using large language models (LLMs).\nLangChain provides abstractions for working with prompts, chains, memory, and agents, making it easier to build complex LLM-based systems.\nThe framework supports integration with various vector databases like FAISS and Chroma for semantic retrieval.'),
 Document(metadata={'source': 'langchain-rag-dataset.txt'}, page_content='LangChain enables Retrieval-Augmented Generation (RAG) by allowing developers to fetch relevant context before generating responses.\nMemory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.\nBM25 and vector-based retrieval can be combined in LangChain to support hybrid retrieval strategies

In [4]:
#FAISS Vector Store with HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)
vectorstore

In [6]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5}
)

In [7]:
llm=init_chat_model("groq:llama-3.3-70b-versatile")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x126f183b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x12608af30>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
prompt = PromptTemplate.from_template(
    """
Provide the answer to the question : {input} from the following context : {context}
"""
)

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [12]:
rag_chain = (
    {
        "context":retriever,
        "input": RunnablePassthrough()
    } 
    | RunnablePassthrough.assign(
        answer = prompt | llm | StrOutputParser()
    )
)

In [13]:
response = rag_chain.invoke("How does Langchain support agents and memory ?")
print(response["answer"])
print(response["context"])

LangChain supports agents and memory in the following ways:

1. Agents: LangChain allows LLMs to act as agents that decide which tool to call and in what order during a task. Agents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.

2. Memory: LangChain supports conversational memory using two types:
   - ConversationBufferMemory: This allows models to retain previous interactions, making multi-turn conversations more coherent.
   - ConversationSummaryMemory: This provides summarization memory, enabling models to summarize previous conversations and maintain context.

By providing these features, LangChain enables the development of more complex and interactive LLM-based systems that can engage in multi-turn conversations and make use of various tools and memory mechanisms.
[Document(id='68c7f7ab-1dac-4417-8ac8-33b65f5402fb', metadata={'source': 'langchain-rag-dataset.txt'}, page_content="The 'map-reduce' chain breaks 

In [14]:
response

{'context': [Document(id='68c7f7ab-1dac-4417-8ac8-33b65f5402fb', metadata={'source': 'langchain-rag-dataset.txt'}, page_content="The 'map-reduce' chain breaks up large documents, processes them separately, and then aggregates the outputs.\nThe 'refine' chain iteratively updates an answer by incorporating each new chunk of information.\nLangChain allows LLMs to act as agents that decide which tool to call and in what order during a task.\nLangChain supports conversational memory using ConversationBufferMemory and summarization memory with ConversationSummaryMemory."),
  Document(id='a9f28483-0024-4d63-b02f-048a132ba7a8', metadata={'source': 'langchain-rag-dataset.txt'}, page_content='LangChain is an open-source framework designed to simplify the development of applications using large language models (LLMs).\nLangChain provides abstractions for working with prompts, chains, memory, and agents, making it easier to build complex LLM-based systems.\nThe framework supports integration with 